텍스트를 주면 작동이 되는 분류모형을 만들 수 있어야합니다.
오늘 배운거 중요한 듯

#1.문헌 분류와 감성 분석

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 23.5 MB/s eta 0:00:00


In [ ]:
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt -O ratings_train.txt
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_test.txt -O ratings_test.txt

--2024-04-16 02:03:49--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt [following]
--2024-04-16 02:03:49--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  77.0MB/s    in 0.2s    



In [ ]:
import pandas as pd
pd.read_csv('ratings_test.txt', sep='\t').head(5)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')#탭으로 값이 구분된 파일
print(train_df.shape)
train_df = train_df.iloc[0:1500, :]

train, test = train_test_split( train_df, test_size=0.2)
train_df.head(7)

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0


In [ ]:
from konlpy.tag import Okt
twitter = Okt() #open korean text 형태소분석기
def tw_tokenizer(text):
    tokens_ko = twitter.nouns(text) #명사만 추출
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer   ################# 벡터라이져 두개
from sklearn.feature_extraction.text import CountVectorizer    ################# 벡터라이져 두개

tfidf_vect = CountVectorizer(tokenizer=tw_tokenizer,
                             stop_words=["그", "나", "이", "게" ],
                             min_df=10 )
tfidf_vect.fit(train['document'])  #train의 명사를 뽑은 것까지                        # train의 명사를 뽑은 것까지
tfidf_matrix_train = tfidf_vect.transform(train['document'])                           # DTM 뽑기
tfidf_matrix_test = tfidf_vect.transform(test['document'])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
tfidf_vect.vocabulary_.keys()

dict_keys(['배우', '줄', '영화', '가장', '여자', '쓰레기', '점', '감독', '끝', '정말', '감동', '소재', '액션', '전개', '가슴', '최고', '거', '완전', '한번', '일본', '명작', '평점', '볼', '만', '연기', '좀', '난', '것', '꼭', '처음', '인생', '그냥', '뿐', '작품', '보고', '임', '진짜', '최악', '스토리', '안', '더', '기', '중', '느낌', '남자', '애', '듯', '음악', '정도', '사랑', '사람', '지금', '드라마', '무슨', '설정', '이건', '기대', '연출', '왜', '말', '내용', '이야기', '주인공', '때', '절대', '못', '개봉', '하나', '보기', '재미', '개', '조금', '수', '생각', '제일', '별로', '뭐', '함', '걸', '진심', '마지막', '자체', '도', '시간', '인간', '시리즈', '내', '마음', '돈', '다시', '장면', '또', '뭔가', '내내', '실망', '기억', '건', '속', '역시'])

In [ ]:
tfidf_vect.vocabulary_

{'배우': 39,
 '줄': 87,
 '영화': 64,
 '가장': 1,
 '여자': 60,
 '쓰레기': 56,
 '점': 80,
 '감독': 2,
 '끝': 15,
 '정말': 82,
 '감동': 3,
 '소재': 49,
 '액션': 59,
 '전개': 78,
 '가슴': 0,
 '최고': 93,
 '거': 6,
 '완전': 65,
 '한번': 97,
 '일본': 72,
 '명작': 34,
 '평점': 95,
 '볼': 43,
 '만': 32,
 '연기': 62,
 '좀': 85,
 '난': 16,
 '것': 9,
 '꼭': 14,
 '처음': 92,
 '인생': 71,
 '그냥': 10,
 '뿐': 44,
 '작품': 75,
 '보고': 41,
 '임': 73,
 '진짜': 91,
 '최악': 94,
 '스토리': 52,
 '안': 57,
 '더': 23,
 '기': 11,
 '중': 88,
 '느낌': 21,
 '남자': 17,
 '애': 58,
 '듯': 27,
 '음악': 67,
 '정도': 81,
 '사랑': 46,
 '사람': 45,
 '지금': 89,
 '드라마': 26,
 '무슨': 36,
 '설정': 48,
 '이건': 68,
 '기대': 12,
 '연출': 63,
 '왜': 66,
 '말': 33,
 '내용': 20,
 '이야기': 69,
 '주인공': 86,
 '때': 28,
 '절대': 79,
 '못': 35,
 '개봉': 5,
 '하나': 96,
 '보기': 42,
 '재미': 77,
 '개': 4,
 '조금': 84,
 '수': 51,
 '생각': 47,
 '제일': 83,
 '별로': 40,
 '뭐': 37,
 '함': 98,
 '걸': 8,
 '진심': 90,
 '마지막': 31,
 '자체': 74,
 '도': 24,
 '시간': 53,
 '인간': 70,
 '시리즈': 54,
 '내': 18,
 '마음': 30,
 '돈': 25,
 '다시': 22,
 '장면': 76,
 '또': 29,
 '뭔가': 38,
 '내내': 19,

In [ ]:
#RF 이용하여 감성 분석 Classification 수행.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#n_estimators: RF에 들어가는 tree의 개수
rf = RandomForestClassifier(random_state=0, n_estimators=400)
rf.fit(tfidf_matrix_train , train['label']  )  #모델링
preds1 = rf.predict(tfidf_matrix_test) #테스트셋으로 감성예측
print('Accuracy: ',accuracy_score(test['label'],preds1))#정답율

Accuracy:  0.6266666666666667


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test['label'],preds1))

              precision    recall  f1-score   support

           0       0.61      0.76      0.68       156
           1       0.65      0.49      0.56       144

    accuracy                           0.63       300
   macro avg       0.63      0.62      0.62       300
weighted avg       0.63      0.63      0.62       300



#2. 감성 스코어

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')#탭으로 값이 구분된 파일
print(train_df.shape)
train_df = train_df.iloc[10000:13000, :]

train, test = train_test_split( train_df, test_size=0.2)
train_df.head(7)

(150000, 3)


,id,document,label
10000,9246662,최고의 2d영화가 아닌가싶으다....!,1
10001,3207978,카리스마작렬~역대퍼니셔시리즈1.2.3中가장좋은액션수작!!,1
10002,8117078,음..별로다..정말로....,0
10003,9782055,"하....정말 말이 안나오는 영화. 내용, 연출, 카메라, 연기까지 뭐 하나 제대로...",0
10004,8384815,"액션보다는 드라마....누미라파스, 콜린패럴의 감정연기가 정말 굳인 영화",1
10005,7252400,잔잔한 감동을 불러 일으키는 영화... 그래 아직 늦지 않았다~!!,1
10006,10059624,아 정말 이런 힐링영화를 상영하는 영화관이 한 두곳밖에 없다는게 정말 안타깝고 대한...,1


In [ ]:
train_df.label.value_counts()

label
0    1526
1    1474
Name: count, dtype: int64

In [ ]:
train_pos = train[train.label==1]
train_pos.label.value_counts()

label
1    1180
Name: count, dtype: int64

In [ ]:
from konlpy.tag import Okt
twitter = Okt() #open korean text 형태소분석기
def tw_tokenizer(text):
    tokens_ko = twitter.nouns(text) #명사만 추출
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer,
                             stop_words=["그", "나", "이", "게","건",'거' ],
                             min_df=5 )
tfidf_vect.fit(train_pos['document'])  #train의 명사를 뽑은 것까지
tfidf_matrix_train = tfidf_vect.transform(train_pos['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
list( tfidf_vect.vocabulary_.keys() )[1:10]

['나름', '감동', '캐릭터', '후회', '제', '영화', '뭔가', '때', '두']

In [ ]:
tfidf_vect.vocabulary_

In [ ]:
# tfidf_vect. .....(.....은 뭐 여러가지있음)

In [ ]:
tfidf_matrix_train.toarray()

In [ ]:
tfidf_vect.get_feature_names_out()

In [ ]:
import pandas as pd

dtm_tfidf_df = pd.DataFrame(tfidf_matrix_train.toarray(), columns=tfidf_vect.get_feature_names_out(), index = train_pos.index)
print(dtm_tfidf_df.sum().sort_values(ascending=False))
words = dtm_tfidf_df.max().sort_values(ascending=False)[100:130]  #[ 30:100 ], [100:130], [150:]
words.index.tolist()

In [ ]:
dtm_tfidf_df

In [ ]:
dtm_tfidf_df.mean()

In [ ]:
dtm_tfidf_df.sum()

In [ ]:
print(dtm_tfidf_df.sum().sort_values(ascending=False))  # ascending=False : 오름차순이 폴스라서 내림차순

영화    126.067688
최고     44.641201
정말     42.733778
감동     33.189583
진짜     33.061170
         ...    
소설      2.290345
게임      2.280836
어른      2.246871
이해      2.144152
해       2.120498
Length: 220, dtype: float64


In [ ]:
tf_vect = CountVectorizer(tokenizer=tw_tokenizer,
                             stop_words=["그", "나", "이", "게","건",'거' ],
                             vocabulary = words.index.tolist(),
                             min_df=5 )
tf_vect.fit(train['document'])  #train의 명사를 뽑은 것까지
tf_matrix_train = tf_vect.transform(train['document'])
tf_matrix_test = tf_vect.transform(test['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
#RF 이용하여 감성 분석 Classification 수행.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#n_estimators: RF에 들어가는 tree의 개수
rf = RandomForestClassifier(random_state=0, n_estimators=300)
rf.fit(tf_matrix_train , train['label']  )  #모델링
preds1 = rf.predict(tf_matrix_test) #테스트셋으로 감성예측
print('Accuracy: ',accuracy_score(test['label'],preds1))#정답율

Accuracy:  0.5516666666666666


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test['label'],preds1))

              precision    recall  f1-score   support

           0       0.54      0.88      0.67       306
           1       0.63      0.21      0.31       294

    accuracy                           0.55       600
   macro avg       0.58      0.54      0.49       600
weighted avg       0.58      0.55      0.49       600



In [ ]:
#긍정 Score
dtm_tf_df = pd.DataFrame(tf_matrix_train.todense(), columns =words.index.tolist(), index=train.index )

pos_score = pd.DataFrame( dtm_tf_df.sum(axis=1).sort_values(ascending=False), columns=["score"] )
pos_score["id"] = pos_score.index

train["id"] = train.index
sentiment = pd.merge(pos_score, train, on="id")

In [ ]:
sentiment

,score,id,document,label
0,5,10375,일본영화 특유의 유치한 연출과 각본. 하지만 이게 빠지만 일본영화라 할 수 없다. ...,1
1,4,12229,아..이영화 완전 대박...감동적이고 교훈도주네요...마지막엔딩키스신이 아직도 잊혀...,1
2,4,11413,전쟁에 관한 걸작 중의 하나. 테렌스 멜릭 감독의 심호한 철학적 사고가 깊이 베인 ...,1
3,4,10335,보다가 어처구니가 없어서 존쿠삭이나 애드리안브로디 를 삼류로 만드는 감독의 능력 일...,0
4,4,11661,애들용인데 뭐가 문제냐고 하겠지만 내가 어릴 때 봐도 결말보고 유치해서 손발 오그라...,0
...,...,...,...,...
2395,0,11032,정말 재밌었던 영화... 주제곡도 너무 멋있었던...,1
2396,0,11469,너무재미있게보고있었는데 이건 너무하네요..어의없게 어떻게 윤재한테 다시 갈수가있어요...,0
2397,0,11342,정말정말 재미있었는데ㅎㅋㅋㅋㅋㅋㅋㅋㅋㅋ,1
2398,0,10435,민효린씨가 계속 앉아만 있어도 이것보단 재미있을 듯,0
